<img src="http://oci02.img.iteso.mx/identidad_de_instancia_2018/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg">

<center> <font color= #004A94> <font size = 6> Lic. Ingeniería Financiera </font> <br> <br> <font color= #047CFB> <font size = 4>I.F. Alejandra Cortés Soto - if708833@iteso.mx </font>

<center><font color= #004A94> <font size = 6> Microestructura y Sistemas de Trading </font> <br> <br>

<center><font color=#42c29b><font size=5> Laboratorio 0: Repaso de Python </font> <br> <br>

<center> <font color= #047CFB> <font size = 4> Primavera 2020 </font>  |  <font color= #047CFB> <font size = 4> Repositorio: <a href='https://github.com/'>https://github.com/alecortees22/LAB_0_ACS</a></font>

# Resumen

Este laboratorio se basó en el uso de funciones específicas del lenguaje de programación python, con la finalidad de aprender a investigar en fuentes externas y de ayuda, así como recordar los aprendizajes obtenidos en cursos anteriores respecto a dicho lenguaje y saber interpretar los datos obtenidos en tu análisis y obtener resultados de apoyo en la toma de decisiones.
Utilizamos la herramienta de las gráficas de vela para el estudio de los datos históricos.

# Codigo

In [3]:
# Paso 1: importar todas las paqueterías necesarias para que tu código funcione correctamente
import funciones as fn #Utilizamos como apoyo las funciones proporcionadas por el profesor, las importamos de ese archivo
import visualizaciones as vs
import pandas as pd
import numpy as np
import plotly.graph_objects as go #Rama de la paquetería plotly que te permite graficar datos 
import plotly.express as px


In [4]:
# Paso 2: descargar precios utilizando la herramienta OANDA
# Este código nos permite descargar los precios según los parámetros que nosotros indiquemos con la ayuda de la herramienta OANDA
OA_Ak = 'db61d16ed80a943c9b65769aea7b75e8-dec8e1238889457a886b69e85640efec' # token personal generado en OANDA
OA_In = "EUR_USD"  # Utilizamos como instrumento de estudio la pardidad de EUR-USD
OA_Gn = "D"  # Granularidad de velas
# Nosotros determinamos los parámetros específicos para la descarga de históricos
fini = pd.to_datetime("2019-01-06 00:00:00").tz_localize('GMT')  # Fecha inicial
ffin = pd.to_datetime("2019-12-06 00:00:00").tz_localize('GMT')  # Fecha final

df_pe = fn.f_precios_masivos(p0_fini=fini, p1_ffin=ffin, p2_gran=OA_Gn,
                             p3_inst=OA_In, p4_oatk=OA_Ak, p5_ginc=4900)

In [5]:
# Paso 3: graficar los datos históricos descargados en OANDA utilizando la herramienta plotly
vs_grafica1 = vs.g_velas(p0_de=df_pe.iloc[0:120, :]) #Utilizamos la gráfica de velas como herramienta visual de los históricos
vs_grafica1.show()

In [6]:
# Paso 4: iniciamos el conteno de velas
# Primero utilizamos la herramienta de hora para determinar la hora en la que fueron descargados los históricos
df_pe['Hora'] = list(df_pe['TimeStamp'][i].hour for i in range(0, len(df_pe['TimeStamp'])))  #Creamos una nueva columna de hora
# Ahora utilizamos la herramienta de día de la semana,la cual nos va a permitir sacar conclusiones al final del ejercicio
# -- 0A.2: Dia de la semana.
df_pe['Dia'] = list(df_pe['TimeStamp'][i].weekday() for i in range(0, len(df_pe['TimeStamp'])))
# Calculamos la diferencia entres los precios de inicio y de cierre para determinar la amplitud de las velas
# -- CO: Amplitud de velas (close - open).
closes = pd.DataFrame(float(i) for i in df_pe['Close'])
opens = pd.DataFrame(float(i) for i in df_pe['Open'])
df_pe['CO'] = (closes - opens) #Hacemos una columna para colocar esa diferencia y estudiar los cambios del precio entre inicio y fin del dia

In [7]:
# Inicio de resultados desarrollados individualmente
# Utilizamos la columna de TimeStamp para rescatar el mes donde se obtuvieron los datos para futuro análisis de los mismos
# -- 01: Mes de la vela.
df_pe['Mes'] = pd.DataFrame(i.month for i in df_pe['TimeStamp']) #Colocamos los datos en la nueva columna mes
# Al momento de querer calcular la sesión de la vela tuvimos que darle valores a la hora en la que sucedió el movimiento 
# para así poder determinar la sesión específica a la que pertenece el movimiento
# -- 02: Sesion de la vela.
# Utilizamos el apoyo de la función tuple
condiciones = [(lambda i: i == 22 or i == 23 or i == 0 or i == 1 or i == 2 or i == 3 or i == 4 or i == 5 or i == 6 or
                          i == 7, 'ASIA'),
               (lambda i: i == 8, 'ASIA_EUROPA'),
               (lambda i: i == 9 or i == 10 or i == 11 or i == 12, 'EUROPA'),
               (lambda i: i == 13 or i == 14 or i == 15 or i == 16, 'EUROPA_AMERICA'),
               (lambda i: i == 17 or i == 18 or i == 19 or i == 20 or i == 21, 'AMERICA')]


# Ahora creamos una función que nos permita utilizar la función de tupla con las condiciones establecidas y los datos de la sección hora
def apply_condiciones(i):
    for condicion, reemplazo in condiciones:
        if condicion(i):
            return reemplazo
    return i


# Por último creamos la columna de sesiones y asignamos los datos obtenidos en base a la función creada
df_pe['Sesion'] = list(map(apply_condiciones, df_pe['Hora']))


In [8]:
# Aqui utilizamos la amplitud de vela CO ya calculada y la ponemos en porcentaje en puntos para después poder compararlo con el día de la semana 
# -- 03: Amplitud OC esperada de vela para cualquier dia de la semana (Dist de Freq).
df_pe['OC'] = (closes - opens) * 10000 #lo multiplicamos por 10000 para ponerlo en PIP
# Hacemos el mismo proceso pero ahora con la diferencia del precio más alto y bajo presentado en la sesión
# -- 04: Amplitud HL esperada de vela para cualquier dia de la semana (Dist de Freq).
high = pd.DataFrame(float(i) for i in df_pe['High'])
low = pd.DataFrame(float(i) for i in df_pe['Low'])
df_pe['HL'] = (high - low) * 10000 # También lo ponemos en PIP 
# Ahora determinamos el sentido que tiene cada vela para así poder predecir tendencias 
# -- 05: Sentido de la vela (Alcista o Bajista)
sentido = (lambda opens, closes: 'ALCISTA' if closes >= opens else 'BAJISTA') #Si el cierre es mayor al precio inicial nos vamos a tendencia alcista, de lo contrario bajita
df_pe['Sentido'] = pd.DataFrame(sentido(df_pe['Open'][i], df_pe['Close'][i]) for i in range(len(df_pe['Open']))) #Metemos los datos en una columna

In [10]:
#Utilizamos la herramienta de ventanas móviles para estudiar los cambios máximos esperados entre los precios del activo en una sesión
# a base de los datos históricos obtenidos y escenciales para la ciencia de datos 
# -- 07: Ventanas móviles de volatilidad
# En este caso utilizamos la función rolling que nos permite elegir una ventana móvil con cierto número de muestras para posteriormente
# aplicar alguna medida estadística, en este caso utilizamos la función max para obtener el máximo valor de la ventana móvil indicada
df_pe['volatilidad_5'] = (df_pe['HL'].rolling(5).max()) #Ventana de 5 
df_pe['volatilidad_25'] = (df_pe['HL'].rolling(25).max()) #Ventana de 25
df_pe['volatilidad_50'] = (df_pe['HL'].rolling(50).max()) #Ventana de 50

# Por último, decidimos graficar el promedio de las volatilidades obtenidas en la ventana móvil de 5 respecto a cada día de la semana

# -- 08: Gráfica con Plotly
c = df_pe.groupby(['Dia'])['volatilidad_5'].mean() #primero agrupamos los datos de dicha ventana con los días de la semana y le sacamos promedio
#Decidimos utilizar una herramienta de plotly llamada scatter que te permite graficar los puntos entre dos variables
fig = px.scatter(x=[0, 1, 2, 3, 4, 6], y=c, labels={'x': 'Dia de la semana', 'y': 'Promedio volatilidad (5)'},
                 title='Volatilidad en los días de la semana')
fig.show()
#Obtenemos la gráfica y notamos que el día más volátil es el día 3, mitad de semana y que inicio de semana presenta una volatilidad
# baja respecto a los demás días 